In [ ]:
import pandas as pd
import numpy as np
import sklearn
import re
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statistics

In [ ]:
#データセット読み込み
y = pd.read_csv("train_y.csv", encoding="utf-8", engine='python')
x = pd.read_csv("train_x (1).csv", encoding="utf-8", engine='python')

#目的変数と説明変数に分ける
xy = pd.concat([y, x], axis=1)
y2 = xy["応募数 合計"]
x = xy.drop(["応募数 合計", "お仕事No."], axis=1)

In [ ]:
#新たな特徴量を生成

#給与/交通費　備考のリストに含まれる文字列から給料の値を抽出する
result = []

for i in range(len(x)):
  ans = ['nan', 'nan']
  if type(x["給与/交通費\u3000備考"][i]) == str:
    ans = re.findall(r"\d+", x["給与/交通費\u3000備考"][i]) 
    if len(ans) == 0:
      ans = ['nan', 'nan']
  
  result.append(ans)
  #NaNは中央値で埋める
  x["給与/交通費\u3000備考"][i]   = float(result[i][0]) * 10000 + float(result[i][1])
  

#k = pd.DataFrame( x["給与/交通費\u3000備考"])
#k = k.fillna(statistics.median(x["給与/交通費\u3000備考"]))
#x["給与/交通費\u3000備考"] = k
#x["給与/交通費\u3000備考"]

#給与のリストに含まれる文字列から給料の値(単位: 万)を抽出する

result = []

for i in range(len(x)):
  ans = ['nan']
  if type(x['（紹介予定）年収・給与例'][i]) == str:
    ans = re.findall(r"\d+", x['（紹介予定）年収・給与例'][i])
    if len(ans) == 0:
      ans = ['nan']

  result.append(ans)
  x['（紹介予定）年収・給与例'][i] = float(result[i][0]) 

k = pd.DataFrame( x["（紹介予定）年収・給与例"])
k = k.fillna(statistics.median(x["（紹介予定）年収・給与例"]))
x["（紹介予定）年収・給与例"] = k


#期間・時間\u3000勤務時間から仕事の開始時間と終了時間を抽出

result = []
start = []
end = []

for i in range(len(x)):
  ans = ['nan','nan', 'nan','nan']
  if type(x["期間・時間\u3000勤務時間"][i]) == str:
    ans = re.findall(r"\d+",x["期間・時間\u3000勤務時間"][i])
    if len(ans) == 0:
      #何も抽出できなかった場合は次のもので埋める
      ans =  ['nan','nan', 'nan','nan']
  
  a = float(ans[0]) + float(ans[1]) / 60
  b = float(ans[2]) + float(ans[3]) / 60

  start.append(a)
  end.append(b)
  
jikan = pd.DataFrame((start,end)).T
jikan.columns = ["開始時刻","終了時刻"]

jikan["開始時刻"] = jikan.fillna(statistics.median(jikan["開始時刻"]))
jikan["終了時刻"] = jikan.fillna(statistics.median(jikan["終了時刻"]))
jikan

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,開始時刻,終了時刻
0,9.5,9.5
1,9.0,9.0
2,9.0,9.0
3,9.0,9.0
4,9.0,9.0
...,...,...
15848,8.5,8.5
15849,10.0,10.0
15850,10.0,10.0
15851,9.0,9.0


In [ ]:
#質的データの中からダミー変数にした時に必要以上に多くなりそうな変数とそうでない変数に場合分け

name = x.columns
x_dum = []
x_dro = []
t = 10

for i in range(0, len(x.columns)):
  for j in range(0, len(x)):
    if type(x[name[i]][j]) == str:
      dum = pd.get_dummies(x[name[i]])
      if len(dum.columns) < t:
        x_dum.append(name[i]) #ダミーが多くなる変数
      else:
        x_dro.append(name[i]) #ダミーが小さくなる変数
      break

#データを整形


x2 = x.drop(x_dro, axis=1) 
dum = pd.get_dummies(x[x_dum])
#結果的にダミー変数を採用しない方が誤差が小さかった.
x2 = x2.drop(x_dum, axis=1)
x2 = pd.concat((x2, jikan), axis=1)

In [ ]:
na = ["開始時刻","終了時刻", "給与/交通費　備考", "給与/交通費　給与上限","（紹介予定）年収・給与例","給与/交通費　給与下限", "給与/交通費　交通費"]

In [ ]:
#お金と時間に関わる変数のNaNを中央値で埋める
for n in na:
  print(n)
  x2[n] = x2[n].fillna(statistics.median(x2[n]))

開始時刻
終了時刻
給与/交通費　備考
給与/交通費　給与上限
（紹介予定）年収・給与例
給与/交通費　給与下限
給与/交通費　交通費


In [ ]:
x2["（紹介予定）年収・給与例"] = x2["（紹介予定）年収・給与例"].fillna(250) 

In [ ]:
#訓練データとテストデータに分ける

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x2, y2, test_size=0.2, random_state=0)
X_train.head(1)

,職場の様子,（紹介予定）入社後の雇用形態,勤務地 最寄駅3（駅名）,勤務地固定,休日休暇(月曜日),応募先 名称,（派遣先）配属先部署 男女比 男,勤務地 最寄駅3（沿線名）,（派遣先）勤務先写真コメント,勤務地 最寄駅3（分）,無期雇用派遣,未使用.14,（派遣以外）応募後の流れ,（派遣先）概要 従業員数,電話応対なし,大手企業,週払い,週1日からOK,交通費別途支給,（派遣先）配属先部署 人数,固定残業制 残業代 下限,残業月20時間以上,職種コード,1日7時間以下勤務OK,ミドル（40〜）活躍中,ルーティンワークがメイン,未使用.11,短時間勤務OK(1日4h以内),駅から徒歩5分以内,フリー項目 内容,先輩からのメッセージ,対象者設定 年齢下限,未使用.10,学校・公的機関（官公庁）,土日祝のみ勤務,Wordのスキルを活かす,未使用.8,経験必須,固定残業制 残業代に充当する労働時間数 下限,給与/交通費 給与支払区分,...,給与 経験者給与下限,勤務地 最寄駅1（駅からの交通手段）,学生歓迎,紹介予定派遣,固定残業制 残業代 上限,未使用.19,給与 経験者給与上限,シフト勤務,経験者優遇,週4日勤務,未経験OK,土日祝休み,給与/交通費 交通費,未使用.21,待遇・福利厚生,シニア（60〜）歓迎,ベンチャー企業,少人数の職場,仕事写真（下） 写真3 コメント,新卒・第二新卒歓迎,休日休暇(金曜日),産休育休取得事例あり,扶養控除内,給与/交通費 給与下限,対象者設定 性別,WEB登録OK,応募先 備考,オフィスが禁煙・分煙,応募先 所在地 市区町村,仕事写真（下） 写真1 コメント,勤務地 市区町村コード,未使用.13,勤務地 最寄駅2（分）,応募拠点,給与/交通費 備考,残業なし,これまでの採用者例,（派遣先）概要 勤務先名（フリガナ）,開始時刻,終了時刻
13515,3,NaN,NaN,1,0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,5.0,NaN,1,22030,0,1,1,NaN,0,0,NaN,NaN,0,NaN,0,0,1,NaN,NaN,NaN,3,...,NaN,2.0,NaN,0,NaN,NaN,NaN,0,1,0,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,2850,1,1,NaN,1,NaN,NaN,111,NaN,NaN,NaN,463838.0,0,NaN,NaN,8.75,8.75


In [ ]:
#分散が0となる (すべて値が同じ) データは変数から除外する

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0) 
sel.fit(X_train)

X_train = X_train.loc[:, sel.get_support()]
X_test = X_test.loc[:, sel.get_support()]

X_train.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_variance_threshold.py:77: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_variance_threshold.py:85: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_variance_threshold.py:88: RuntimeWarning: invalid value encountered in less_equal
  (self.variances_ <= self.threshold)):
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_variance_threshold.py:99: RuntimeWarning: invalid value encountered in greater
  return self.variances_ > self.threshold


(12682, 72)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate,cross_val_predict, StratifiedKFold

#GBDTモデルを作成
reg = xgb.XGBRegressor()

#フォールド数5のクロスヴァリデーションを行いながら
#パラメータをグリッドサーチで最適化する

reg_cv = GridSearchCV(reg, {'max_depth': [2, 6, 7], 'n_estimators': [400, 200, 100]}, verbose=1, cv=5, scoring="neg_mean_squared_error")
reg_cv.fit(X_train, y_train)
print(reg_cv.best_params_, reg_cv.best_score_)

In [ ]:
#最適化されたパラメータで再びモデル作成
reg = xgb.XGBRegressor(**reg_cv.best_params_)
reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

#テストデータから応募者合計を予測
ans = reg.predict(X_test)
np.sqrt(mean_squared_error(ans, y_test))

0.589931808067223

In [ ]:
#課題用のテストデータを読み込み
test = pd.read_csv("test_x.csv", encoding = "utf-8", engine="python")
jobnum = test["お仕事No."]
test = test.drop("お仕事No.", axis=1)

In [ ]:
test.shape#確認用

(3391, 211)

In [ ]:
#新たな特徴量を生成

#給与/交通費　備考のリストに含まれる文字列から給料の値を抽出する
result = []

for i in range(len(test)):
  ans = ['nan', 'nan']
  if type(test["給与/交通費\u3000備考"][i]) == str:
    ans = re.findall(r"\d+", test["給与/交通費\u3000備考"][i]) 
    if len(ans) == 0:
      ans = ['nan', 'nan']
  
  result.append(ans)
 
  test["給与/交通費\u3000備考"][i]   = float(result[i][0]) * 10000 + float(result[i][1])
  
#NaNは中央値で埋める
k = pd.DataFrame(test["給与/交通費\u3000備考"])
k = k.fillna(statistics.median(test["給与/交通費\u3000備考"]))
test["給与/交通費\u3000備考"] = k


#給与のリストに含まれる文字列から給料の値(単位: 万)を抽出する

result = []

for i in range(len(test)):
  ans = ['nan']
  if type(test['（紹介予定）年収・給与例'][i]) == str:
    ans = re.findall(r"\d+", test['（紹介予定）年収・給与例'][i])
    if len(ans) == 0:
      ans = ['nan']

  result.append(ans)
  test['（紹介予定）年収・給与例'][i] = float(result[i][0]) 

k = pd.DataFrame(test["（紹介予定）年収・給与例"])
k = k.fillna(statistics.median(test["（紹介予定）年収・給与例"]))
test["（紹介予定）年収・給与例"] = k


#期間・時間\u3000勤務時間から仕事の開始時間と終了時間を抽出

result = []
start = []
end = []

for i in range(len(test)):
  ans = ['nan','nan', 'nan','nan']
  if type(test["期間・時間\u3000勤務時間"][i]) == str:
    ans = re.findall(r"\d+",test["期間・時間\u3000勤務時間"][i])
    if len(ans) == 0:
      #何も抽出できなかった場合は次のもので埋める
      ans =  ['nan','nan', 'nan','nan']
  
  a = float(ans[0]) + float(ans[1]) / 60
  b = float(ans[2]) + float(ans[3]) / 60

  start.append(a)
  end.append(b)
  
jikan = pd.DataFrame((start,end)).T
jikan.columns = ["開始時刻","終了時刻"]

jikan["開始時刻"] = jikan.fillna(statistics.median(jikan["開始時刻"]))
jikan["終了時刻"] = jikan.fillna(statistics.median(jikan["終了時刻"]))
jikan

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,開始時刻,終了時刻
0,8.5,8.5
1,9.0,9.0
2,9.0,9.0
3,9.0,9.0
4,9.0,9.0
...,...,...
3386,9.0,9.0
3387,9.0,9.0
3388,10.0,10.0
3389,9.0,9.0


In [ ]:
test2 = pd.concat((test, jikan), axis=1)

In [ ]:
#課題用テストデータの説明変数の種類を訓練データのそれに合わせる. 
test2 = test2[X_train.columns]

In [ ]:
#お金と時間に関わる変数のNaNを中央値で埋める
for n in na:
  print(n)
  test2[n] = test2[n].fillna(statistics.median(test2[n]))

開始時刻
終了時刻
給与/交通費　備考
給与/交通費　給与上限
（紹介予定）年収・給与例
給与/交通費　給与下限
給与/交通費　交通費


In [ ]:
#課題用テストデータで応募者合計を予測する
pred = reg.predict(test2)

In [ ]:
#データフレームにまとめる
ans = pd.DataFrame((jobnum, pred), index=["お仕事No.", "応募数 合計"]).T

In [ ]:
#csvファイルに書き込む
ans.to_csv("di.csv", encoding = "utf-8", index=False)

In [ ]:
ans #確認用

,お仕事No.,応募数 合計
0,2.203557e+09,2.158051
1,2.203515e+09,4.113380
2,2.203547e+09,3.425374
3,2.203545e+09,3.572912
4,2.203558e+09,2.214189
...,...,...
3386,2.203538e+09,0.070693
3387,2.203544e+09,-0.224195
3388,2.203557e+09,0.506288
3389,2.203506e+09,-0.205845
